# VIsual Attention Project - FIGRIM Dataset
Valerijan Matvejev and Hana Ujcic

In [ ]:
%matplotlib inline

import os
import glob
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from scipy.ndimage import gaussian_filter
from scipy.io import loadmat
from matplotlib import cm
from scipy.ndimage import gaussian_filter
import nbimporter
from os import listdir
from os.path import isfile, join


from Achanta import Achanta

#path to dataset
PATH_DATA = "C:/Master/PC/VAproject/FIGRIM/FIGRIM" # path to the dataset
PATH_DEEP = "C:/Master/PC/VAproject/saliency/results/images" # path to images generated by MSINet


# Experiment-dependant constants
RESO_X = 1000
RESO_Y = 1000
FACTOR_X = RESO_X/270
FACTOR_Y = RESO_Y/203

In [ ]:
def range_normalize(x):
    """Normalizes x to [0, 1]"""
    x = (x - x.min()) / (x.max() - x.min())
    return x

def sum_normalize(x):
    """Normalizes x so that it sums to 1"""
    return x / x.sum()


def std_normalize(x):
    return (x - np.mean(x)) / np.std(x)

def log_density(saliencyMap, eps=np.spacing(1.0)):
    """Transforms a non probabilistic saliency map into a log density for
    further metric computation.

    Arguments:
        saliencyMap: Grayscale saliency map.
    """
    saliencyMap = saliencyMap - saliencyMap.min()
    saliencyMap += eps
    saliencyMap /= saliencyMap.sum()
    return np.log(saliencyMap)

And now, the metrics !

def AUC_Borji(saliencyMap, fixationMap, Nsplits=100, stepSize=0.1):
    """
    This measures how well the saliencyMap of an image predicts the ground
    truth human fixations on the image.

    ROC curve created by sweeping through threshold values at fixed step size
    until the maximum saliency map value.
    True positive (tp) rate corresponds to the ratio of saliency values
    above threshold at fixation locations to the total number of fixation
    locations.
    False positive (fp) rate corresponds to the ratio of saliency map
    values above threshold at random locations to the total number of random
    locations (as many random locations as fixations, sampled uniformly from
    ALL IMAGE PIXELS), averaging over Nsplits number of selections of
    random locations.

    :param saliencyMap: numpy array, saliency map
    :param fixationMap: numpy array, fixation map (binary matrix)
    :param Nsplits: int, number of random splits
    :param stepSize: float, size of the step for sweeping through saliency map

    :return score: float, AUC Borji score
    :return fpr: float, false positive rate
    :return tpr: float, true positive rate
    """
    
    saliencyMap = range_normalize(saliencyMap)

    S = saliencyMap.flatten()
    F = fixationMap.flatten()

    Sth = S[F > 0]  # saliency map values at fixation locations
    Nfixations = len(Sth)
    Npixels = len(S)

    # For each fixation, sample Nsplits values from anywhere on the saliency map
    r = np.random.randint(1, Npixels, size=Nfixations * Nsplits)
    randfix = S[r[:]]
    randfix = np.reshape(randfix, (Nfixations, Nsplits))

    # Calculate AUC per random split (set of random locations)
    auc = np.empty(Nsplits)
    auc[:] = np.nan
    for s in range(Nsplits):
        curfix = randfix[:, s]
        allthreshes = np.arange(0, np.amax(np.concatenate([Sth, curfix])) + stepSize, stepSize)
        allthreshes = allthreshes[::-1]
        tpr = np.zeros(len(allthreshes) + 2)
        fpr = np.zeros(len(allthreshes) + 2)
        tpr[0], tpr[-1] = 0, 1
        fpr[0], fpr[-1] = 0, 1
        for i in range(len(allthreshes)):
            thresh = allthreshes[i]
            tpr[i + 1] = (Sth >= thresh).sum() / Nfixations
            fpr[i + 1] = (curfix >= thresh).sum() / Nfixations
        auc[s] = np.trapz(tpr, x=fpr)
    # Average across random splits
    score = np.mean(auc)
    return score, fpr, tpr

def AUC_Judd(saliencyMap, fixationMap, jitter=True):
    """
    This measures how well the saliencyMap of an image predicts the ground
    truth human fixations on the image. ROC curve created by sweeping through
    threshold values determined by range of saliency map values at fixation
    locations.
    Uses a uniform non-fixation distribution, i.e. the full saliency map as
    nonfixations.

    created Tilke Judd, Oct 2009
    updated  Zoya Bylinskii, Aug 2014
    python-version by Dario Zanca, Jan 2017

    true positive (tp) rate correspond to the ratio of saliency map values
    above threshold at fixation locations to the total number of fixation
    locations, false positive (fp) rate correspond to the ratio of
    saliency map values above threshold at all other locations to
    the total number of posible other locations (non-fixated image pixels)

    Arguments
    ---------
        saliencyMap: Saliency map image (grayscale)
        fixationMap: Ground truth fixations (binary image)
        jitter: Whether to add tiny non-zero random constant to all map
            locations to ensure ROC can be calculated robustly
            (to avoid uniform region)

    Returns
    -------
        score: float
            AUC value for the given inputs
    """
    if not np.shape(saliencyMap) == np.shape(fixationMap):
        saliencyMap = cv2.resize(saliencyMap, np.shape(fixationMap)[:2][::-1])

    # jitter saliency maps that come from saliency models that have a lot of
    # zero values.
    # If the saliency map is made with a Gaussian then it does not need to be
    # jittered as the values are varied and there is not a large patch of the
    # same value. In fact jittering breaks the ordering in the small values!
    if jitter:
        # jitter the saliency map slightly to disrupt ties of the same numbers
        saliencyMap = saliencyMap + np.random.random(np.shape(saliencyMap)) / 10**7

    saliencyMap = range_normalize(saliencyMap)

    if np.isnan(saliencyMap).all():
        __logger.debug('NaN saliencyMap')
        score = float('nan')
        return score

    S = saliencyMap.flatten()
    F = fixationMap.flatten()

    # Sal map values at fixation locations
    Sth = S[F > 0]
    Nfixations = len(Sth)
    Npixels = len(S)

    # sort sal map values, to sweep through values
    allthreshes = sorted(Sth, reverse=True)

    tpr = np.zeros((Nfixations + 2))
    fpr = np.zeros((Nfixations + 2))
    tpr[0], tpr[-1] = 0, 1
    fpr[0], fpr[-1] = 0, 1

    for i in range(Nfixations):
        thresh = allthreshes[i]
        # total number of sal map values above threshold
        aboveth = (S >= thresh).sum()
        # ratio sal map values at fixation locations above threshold
        tpr[i + 1] = float(i + 1) / Nfixations
        # ratio other sal map values above threshold
        fpr[i + 1] = float(aboveth - i) / (Npixels - Nfixations)

    score = np.trapz(tpr, x=fpr)
    allthreshes = np.insert(allthreshes, 0, 0)
    allthreshes = np.append(allthreshes, 1)

    return score, fpr, tpr

def kl_divergence(saliencyMap, baselineMap, eps=np.spacing(1.0)):
    """
    Computes the _Image Based_ KL-divergence between two different saliency
    maps when viewed as distributions: it is a non-symmetric measure
    of the information lost when saliencyMap is used to estimate a 'true'
    distribution.

    created: Zoya Bylinskii, Aug 2014\\
    python-version by: Dario Zanca/Pierre-Adrien Fons, 2017-20

    Arguments:
        saliencyMap: Grayscale image of a saliency map.
        baselineMap: Grayscale image of a baseline saliency map to compare
            saliencyMap to.

    Returns:
        KL divergence score (Non negative).
    """

    if saliencyMap.shape != baselineMap.shape:
        saliencyMap = cv2.resize(saliencyMap, np.shape(baselineMap)[:2][::-1])

    if saliencyMap.any():
        saliencyMap = sum_normalize(saliencyMap)
    if baselineMap.any():
        baselineMap = sum_normalize(baselineMap)

    logp_model = np.log(saliencyMap + eps)
    logp_gt = np.log(baselineMap + eps)
    score = baselineMap * (logp_gt - logp_model)

    return score.sum()

def NSS(saliencyMap, fixationMap):
    """
    This finds the normalized scanpath saliency (NSS) of a saliency map.\\
    NSS is the average of the response values at human eye positions in a model
    saliency map that has been normalized to have zero mean and unit standard
    deviation.

    created: Zoya Bylinskii, Aug 2014\\
    python-version by: Dario Zanca/Pierre-Adrien Fons, 2017-20

    Arguments:
        saliencyMap: Saliency map (grayscale).
        fixationMap: Ground truth fixation map (binary matrix).

    Returns:
        NSS score (0 : Chance, >0 : correspondance above Chance, <0 : anti
            correspondance).
    """

    if saliencyMap.shape != np.shape(fixationMap):
        saliencyMap = cv2.resize(saliencyMap, np.shape(fixationMap)[:2][::-1])

    saliencyMap = np.exp(log_density(saliencyMap))

    mean = saliencyMap.mean()
    std = saliencyMap.std()

    value = saliencyMap[fixationMap.astype(bool)]

    value -= mean
    value /= std

    return value.mean()

def CC(saliencyMap1, saliencyMap2):
    """
    Computes the linear correlation coefficient between two different
    saliency maps (also called Pearson's linear coefficient).

    Arguments:
        saliencyMap1: Grayscale Saliency map.
        saliencyMap2: Grayscale Saliency map.

    Returns:
        Linear correlation coefficient ([-1, 1]).
    """

    if saliencyMap1.shape != saliencyMap2.shape:
        saliencyMap1 = cv2.resize(saliencyMap1, np.shape(saliencyMap2)[:2][::-1])

    saliencyMap1 = std_normalize(saliencyMap1)
    saliencyMap2 = std_normalize(saliencyMap2)

    return np.corrcoef(saliencyMap1.reshape(-1), saliencyMap2.reshape(-1))[0, 1]

def similarity(pred_sal, gt_sal):
    """
    This finds the similarity between two different saliency maps when
    viewed as distributions (equivalent to histogram intersection).

    score=1 means the maps are identical
    score=0 means the maps are completely opposite

    "SIM is very sensitive to missing values, and penalizes predictions that
    fail to account for all of the ground truth density."

    Arguments:
        pred_sal: Predicted saliency map (grayscale).
        gt_sal: Ground truth saliency map (grayscale).

    Returns:
        Score of similarity [0, 1] between the two input saliency maps.
    """
    if pred_sal.shape != gt_sal.shape:
        pred_sal = cv2.resize(pred_sal, gt_sal.shape[:2][::-1])

    # (1) first normalize the map values to lie between 0-1 this is done so
    # that models that assign a nonzero value to every pixel do not get an
    # artificial performance boost.
    # (2) then make sure that the map is normalized to sum to 1 so that the
    # maximum value of score will be 1.
    if pred_sal.any():
        pred_sal = range_normalize(pred_sal)
        pred_sal = sum_normalize(pred_sal)

    if gt_sal.any():
        gt_sal = range_normalize(gt_sal)
        gt_sal = sum_normalize(gt_sal)

    diff = np.minimum(pred_sal, gt_sal)
    return np.sum(diff)

In [ ]:
def get_files_all_observers(dir_path, img_name):
    """
    List all eye-tracking data files related to a particular image.
    
    :param dir_path: str, the path to the directory in which the dataset is stored.
    :param img_name: str, the name of the image for which we want to gather eye-tracking data.
                     This must be of the form img.trn.xxx or img.tst.xxx
    :return list_all_files: list, containing the paths to all the relevant eye-tracking files.
    """
    list_all_files = glob.glob(os.path.join(dir_path, 'Data/**/*' + img_name + '.*'), recursive=True)
    assert list_all_files, "List of eye-tracking files seem to be empty. Check directory or image name."
    return list_all_files

In [ ]:
def create_fixmap(list_obs_files, img_w, img_h, factor_x, factor_y, t_begin=0, t_end=15):
    """
    Create a fixation map based on raw eye-tracking data.
    
    :param list_obs_files: list, a list of paths to the eye-tracking files of each observer for the considered image
    :param img_w: int, the width of the image, in pixels
    :param imw_h: int, the height of the image, in pixels
    :param factor_x: float, the ratio of the horizontal resolution to the horizontal size of the screen used in the eye-tracking experiment
    :param factor_y: float, the ratio of the vertical resolution to the vertical size of the screen used in the eye-tracking experimen
    :param t_begin: float, the start of the time slice to consider, in s.
    :param t_end: float, the end of the time slice to consider, in s.
    :return fixmap: numpy array, the fixation map.
    """
    # First, let's initialize the fixation map.
    fixmap = np.zeros((img_h, img_w))
    sigma_filter = 5
    for obs in list_obs_files:
        #load fixation points
        mat_data = loadmat(obs)
        data_key = mat_data['fixLocs']
        
        #create fixation map by convolving a gaussian filter
        for x in range(len(data_key)):
            for y in range(len(data_key[x])):
                if data_key[x][y] != 0 and 0 < x < img_h and 0 < y < img_w:
                    fixmap[x, y] = 1
        fixmap = gaussian_filter(fixmap, sigma=sigma_filter)
        fixmap = (fixmap - np.min(fixmap)) / (np.max(fixmap) - np.min(fixmap))
                    
    
    return fixmap



In [ ]:
def compute_ppda(distance, h_res, v_res, screen_w, screen_h):
    """
    Compute the number of pixels per degree of visual angle based on the experimental conditions.
    
    :param distance: int, the distance between the observer and the screen (in mm)
    :param h_res: int, the horizontal resolution of the screen
    :param v_res: int, the vertical resolution of the screen
    :param screen_w: int, the width of the screen (in mm)
    :param screen_h: int, the height of the screen (in mm)
    :return horizontal_ppda: float, the number of pixel per degree of visual angle
    """    
    ###### TODO ######
    pxl_density_x = h_res / screen_w
    pxl_density_y = v_res / screen_h
    
    # print(str(pxl_density_x) + ' ' + str(pxl_density_y))
    
    d = 2 * distance * math.tan(np.deg2rad(0.5))
    horizontal_ppda = d * ((pxl_density_x + pxl_density_y) / 2)
    
    
    return horizontal_ppda

In [ ]:
def salmap_from_fixmap(fixmap, ppda):
    """
    Generate a visual saliency map, based on the fixation map.
    
    :param fixmap: numpy array, the fixation map
    :param ppda: float, the number of pixels per degree of visual angle
    :return salmap: numpy array, the visual saliency map
    """
    
    sigma = ppda / np.sqrt(2)
    
    salmap = gaussian_filter(fixmap, sigma=sigma)

    ###### TODO ######
    return salmap

In [ ]:
achanta = Achanta()

In [ ]:
#example of one image

files = get_files_all_observers(PATH_DATA, "cockpit/sun_cdtbrxessaoxghds")
fixmap = create_fixmap(files, RESO_X, RESO_Y, FACTOR_X, FACTOR_Y)
img = mpimg.imread(os.path.join(PATH_DATA, "Stimuli/cockpit/sun_cdtbrxessaoxghds.jpg"))
ppda = compute_ppda(415.8, RESO_X, RESO_Y, 270, 203)
salmap = salmap_from_fixmap(fixmap, ppda)
salmap_achanta = achanta.get_salmap(img)
salmap_deep = mpimg.imread(os.path.join(PATH_DEEP, "sun_cdtbrxessaoxghds.jpeg"))

In [ ]:
extent = [0, fixmap.shape[1], 0, fixmap.shape[0]]

fig, axarr = plt.subplots(1, 5, figsize=(15, 15))
axarr[0].imshow(img)
axarr[0].set_title('Image')

axarr[1].imshow(img, cmap = 'viridis', extent = extent)
axarr[1].imshow(fixmap, cmap = 'plasma', alpha = 0.8, extent = extent)
axarr[1].set_title('Fixation points')

axarr[2].imshow(img, cmap = 'viridis', extent = extent)
axarr[2].imshow(salmap, cmap = 'plasma', alpha = 0.8, extent = extent)
axarr[2].set_title('Ground truth')

axarr[3].imshow(img, cmap = 'viridis', extent = extent)
axarr[3].imshow(salmap_achanta, cmap = 'plasma', alpha = 0.8, extent = extent)
axarr[3].set_title('Saliency map Achanta')

axarr[4].imshow(img, cmap = 'viridis', extent = extent)
axarr[4].imshow(salmap_deep, cmap = 'plasma', alpha = 0.8, extent = extent)
axarr[4].set_title('MSINet model')

plt.show()

## Achanta

In [ ]:
# List of folder names
folder_names = ["airport_terminal", "amusement_park", "badlands", "bathroom", "bedroom", "bridge", "castle", "cockpit", "conference_room", "dining_room", "golf_course", "highway", "house", "kitchen", "lighthouse", "living_room", "mountain", "pasture",
                "playground", "skyscraper", "tower"] 

aucBorji = []
klDiv = []
nss = []
cc = []
sim = []


# Define function to calculate metrics
def calculate_metrics_for_folder(folder_name):
    number_of_images = 0
    Borji_mean = 0
    K1_mean = 0
    NSS_mean = 0
    CC_mean = 0
    similarity_mean = 0
    
    # Define the path to the folder
    full_folder_path = os.path.join(PATH_DATA + "/Stimuli", folder_name)

    # Get all image files in the folder
    folder_files = [f for f in os.listdir(full_folder_path) if f.endswith(".jpg")]
    
    # Iterate through each file in the folder
    for file_name in folder_files:
        file_path = os.path.join(full_folder_path, file_name)
        files = get_files_all_observers(PATH_DATA, folder_name + "/" + file_name.split('.')[0])
        if(files is None):
            continue
        # Create fixation map, read image, etc. (use your existing code)
        fixmap = create_fixmap(files, RESO_X, RESO_Y, FACTOR_X, FACTOR_Y)
        img = mpimg.imread(file_path)
        ppda = compute_ppda(415.8, RESO_X, RESO_Y, 270, 203)
        salmap_achanta = achanta.get_salmap(img)
        salmap = salmap_from_fixmap(fixmap, ppda)
        #print("Here")
        # Calculate metrics
        score1b, _, _ = AUC_Borji(saliencyMap=salmap_achanta, fixationMap=fixmap)
        klScore = kl_divergence(saliencyMap=salmap_achanta, baselineMap=salmap)
        nssScore1 = NSS(saliencyMap=salmap_achanta, fixationMap=fixmap)
        ccScore = CC(saliencyMap1=salmap_achanta, saliencyMap2=salmap)
        similarityScore = similarity(pred_sal=salmap_achanta, gt_sal=salmap)

        # Update mean values
        number_of_images += 1
        Borji_mean += score1b
        K1_mean += klScore
        NSS_mean += nssScore1 
        CC_mean += ccScore
        similarity_mean += similarityScore
    
    # Calculate means for the folder
    Borji_mean /= number_of_images
    K1_mean /= number_of_images
    NSS_mean /= number_of_images
    CC_mean /= number_of_images
    similarity_mean /= number_of_images

    aucBorji.append(Borji_mean)
    klDiv.append(K1_mean)
    nss.append(NSS_mean)
    cc.append(CC_mean)
    sim.append(similarity_mean)
        

    # Print metrics with their names
    print(f"Metrics for folder '{folder_name}':")
    print(f"Borji Mean: {Borji_mean}")
    print(f"K1 Mean: {K1_mean}")
    print(f"NSS Mean: {NSS_mean}")
    print(f"CC Mean: {CC_mean}")
    print(f"Similarity Mean: {similarity_mean}")
    print("\n")

# Iterate through each folder and calculate metrics
for folder_name in folder_names:
    calculate_metrics_for_folder(folder_name)



In [ ]:
plt.figure(figsize=(10, 6))
# Plot averages
plt.plot(folder_names, aucBorji, label='AUC Borji', marker='o')
plt.plot(folder_names, klDiv, label='KL divergence', marker='s')
plt.plot(folder_names, nss, label='NSS - normalized scanpath saliency', marker='p')
plt.plot(folder_names, cc, label='linear correlation coefficient', marker='X')
plt.plot(folder_names, sim, label='similarity', marker='D')

# Set labels and title
plt.xlabel('Category')
plt.ylabel('Value')
plt.title('')

# Rotate x-axis labels for better visibility
plt.xticks(rotation=45)

# Show legend
plt.legend()

# Show the plot
plt.show()

## MSINet model
https://github.com/alexanderkroner/saliency

In [ ]:
def plotEvolution(files):
    
    aucBorji = []
    klDiv = []
    nss = []
    cc = []
    sim = []
    
    for file in files:
        
        fixmap = create_fixmap([file], RESO_X, RESO_Y, FACTOR_X, FACTOR_Y)
        ppda = compute_ppda(415.8, RESO_X, RESO_Y, 270, 203)
        gt = salmap_from_fixmap(fixmap, ppda)
        
        file_name_without_extension, file_extension = os.path.splitext(file)
        img_name = file_name_without_extension.split("_")[-1]
        pred = mpimg.imread(os.path.join(PATH_DEEP,"sun_" + img_name + ".jpeg"))
        pred = pred / 255

        score1b, fpr1, tpr1 = Lab_2_VA.AUC_Borji(saliencyMap=pred, fixationMap=fixmap)

        klScore = kl_divergence(saliencyMap=pred, baselineMap=gt)

        nssScore1 = NSS(saliencyMap=pred, fixationMap=fixmap)

        ccScore = CC(saliencyMap1=gt, saliencyMap2=pred)

        similarityScore = similarity(pred_sal=gt, gt_sal=pred)

        aucBorji.append(score1b)
        klDiv.append(klScore)
        nss.append(nssScore1)
        cc.append(ccScore)
        sim.append(similarityScore)
        
    return aucBorji, klDiv, nss, cc, sim


In [ ]:
contents = os.listdir(PATH_DATA + "/Stimuli")

# Use a list comprehension to filter out only directories
directories = [content for content in contents if os.path.isdir(os.path.join(PATH_DATA + "/Stimuli", content))]

aucBorjiScores = []
klDivScores = []
nssScores = []
ccScores = []
simScores = []

for directory in directories:
    list_all_files = glob.glob(os.path.join(PATH_DATA, 'Data/'+ directory + '/' + '*.*'), recursive=True)

    aucBorji, klDiv, nss, cc, sim = plotEvolution(list_all_files)
    
    aucBorjiScores.append(aucBorji)
    klDivScores.append(klDiv)
    nssScores.append(nss)
    ccScores.append(cc)
    simScores.append(sim)
    

In [ ]:
np.savez('aucBorjiScores.npz', *aucBorjiScores)
np.savez('klDivScores.npz', *klDivScores)
np.savez('nssScores.npz', *nssScores)
np.savez('ccScores.npz', *ccScores)
np.savez('simScores.npz', *simScores)

In [ ]:
def compute_statistics(metricScores):

    # Initialize lists to store mean and average values for each array
    mean_values = []
    average_values = []

    for i in range(len(metricScores)):
        # Load each array
        loaded_array = metricScores[i]
        if (loaded_array.size != 0):

            # Compute mean and average for the loaded array
            mean_value = np.mean(loaded_array)
            average_value = np.average(loaded_array)

            # Append the results to the lists
            mean_values.append(mean_value)
            average_values.append(average_value)

    return mean_values, average_values


In [ ]:
borji_mean, borji_avg = compute_statistics(aucBorjiScores)
klDiv_mean, klDiv_avg = compute_statistics(klDivScores)
nss_mean, nss_avg = compute_statistics(nssScores)
cc_mean, cc_avg = compute_statistics(ccScores)
sim_mean, sim_avg = compute_statistics(simScores)

In [ ]:
contents = os.listdir(PATH_DATA + "/Stimuli")
directories = [content for content in contents if os.path.isdir(os.path.join(PATH_DATA + "/Stimuli", content))]
directories.remove('salicon')
print(directories)

In [ ]:
plt.figure(figsize=(10, 6))

# Plot averages
plt.plot(directories, borji_avg, label='AUC Borji', marker='o')
plt.plot(directories, klDiv_avg, label='KL divergence', marker='s')
plt.plot(directories, nss_avg, label='NSS - normalized scanpath saliency', marker='p')
plt.plot(directories, cc_avg, label='linear correlation coefficient', marker='X')
plt.plot(directories, sim_avg, label='similarity', marker='D')

# Set labels and title
plt.xlabel('Category')
plt.ylabel('Value')
plt.title('')

# Rotate x-axis labels for better visibility
plt.xticks(rotation=45)

# Show legend
plt.legend()

# Show the plot
plt.show()